In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import geopandas
import numpy as np

In [2]:
ET.register_namespace('', 'http://www.w3schools.com')
namespace = {'xmlns': 'http://www.w3schools.com'}
tree = ET.parse('data/solution_day_sts_5300RINR.xml')
root = tree.getroot()

# get vehicle locations

In [3]:
class Coordinate(object):
        
    def __init__(self, lon, lat):
        self.lon = lon
        self.lat = lat
        
class Coordinates(object):
    index = 0
    
    def __init__(self):
        Coordinates.index = 0
        self.coords = {}
        
    def update(self, coordinate):
        if coordinate not in self.coords.keys():
            self.coords[coordinate] = Coordinates.index
            Coordinates.index += 1
            
    def get_index(self, coordinate):
        return self.coords[coordinate]


In [4]:
coordinates = Coordinates()

In [5]:
vehicles_element = root.find('xmlns:vehicles', namespace)
vehicle_elements = vehicles_element.findall('xmlns:vehicle', namespace)
for vehicle in vehicle_elements:
    scoord = vehicle.find('xmlns:startLocation/xmlns:coord', namespace)
    slon = scoord.get('x')
    slat = scoord.get('y')
    coordinates.update((slon, slat))
    index = coordinates.get_index((slon, slat))
    id_element = vehicle.find('xmlns:startLocation/xmlns:id', namespace)
    id_element.text = str(index)
    index_element = vehicle.find('xmlns:startLocation/xmlns:index', namespace)
    index_element.text = str(index)
    
    ecoord = vehicle.find('xmlns:endLocation/xmlns:coord', namespace)
    elon = ecoord.get('x')
    elat = ecoord.get('y')
    coordinates.update((elon, elat))
    index = coordinates.get_index((elon, elat))
    id_element = vehicle.find('xmlns:endLocation/xmlns:id', namespace)
    id_element.text = str(index)
    index_element = vehicle.find('xmlns:endLocation/xmlns:index', namespace)
    index_element.text = str(index)

In [6]:
shipments_element = root.find('xmlns:shipments', namespace)
shipment_elements = shipments_element.findall('xmlns:shipment', namespace)
for shipment in shipment_elements:
    shipments_element.remove(shipment)

In [7]:
# initial_routes_element = root.find('xmlns:initialRoutes', namespace)
# root.remove(initial_routes_element)
solutions_element = root.find('xmlns:solutions', namespace)
root.remove(solutions_element)

# Now "others"

In [8]:
def write_coord(parent, location_type, coord, geoid):
    """Writes coordinates to XML.

    jsprit can make graphs with vrp solution, but it requires actual coordinates to do this
    """
    location_element = ET.SubElement(parent, location_type)
    # index is mandatory when solving VRP based on time-distance matrix
    ET.SubElement(location_element, 'index').text = str(geoid)
    # other elements are optional
    ET.SubElement(location_element, 'id').text = str(geoid)
    ET.SubElement(location_element, 'coord', attrib={
        'x': str(coord[0]), 'y': str(coord[1])
    })
    
def write_time_windows(parent, tw_start, tw_end):
    if tw_start is not None and tw_end is not None:
        time_windows_element = ET.SubElement(parent, 'timeWindows')
        time_window_element = ET.SubElement(time_windows_element, 'timeWindow')
        ET.SubElement(time_window_element, 'start').text = str(tw_start)
        ET.SubElement(time_window_element, 'end').text = str(tw_end)
    
def write_shipment_step( parent, shipment_type, coord, geoid, execution_time, tw_start, tw_end):
    shipment_type_element = ET.SubElement(parent, shipment_type)
    write_coord(shipment_type_element, 'location', coord, geoid)
    ET.SubElement(shipment_type_element, 'duration').text = str(execution_time)
    write_time_windows(shipment_type_element, tw_start, tw_end)

In [9]:
persons = pd.read_excel('data/sjöbo_trips_general_sample6421.xlsx')

In [10]:
shipments_element = root.find('xmlns:shipments', namespace)
for person in persons.itertuples():
    shipment_element = ET.SubElement(shipments_element, 'shipment', attrib={'id': str(person.id*1000000)})
    
    shipment_type_element = ET.SubElement(shipment_element, 'pickup')
    scoord = (person.start_longitude, person.start_latitude)
    coordinates.update(scoord)
    sgeoid = coordinates.get_index(scoord)
    write_coord(shipment_type_element, 'location', scoord, sgeoid)
    ET.SubElement(shipment_type_element, 'duration').text = "60"     
    write_time_windows(shipment_type_element, max(person.time - 3600/2, 0), person.time + 3600/2)
    
    ecoord = (person.end_longitude, person.end_latitude)
    coordinates.update(ecoord)
    egeoid = coordinates.get_index(ecoord)
    shipment_type_element = ET.SubElement(shipment_element, 'delivery')
    write_coord(shipment_type_element, 'location', ecoord, egeoid)
    ET.SubElement(shipment_type_element, 'duration').text = "60"     
    write_time_windows(shipment_type_element, max(person.time - 3600/2, 0), person.time + 3600/2 + person.max_ivt)
    
    capacity_element = ET.SubElement(shipment_element, 'capacity-dimensions')
    ET.SubElement(capacity_element, 'dimension', attrib={'index': "0"}).text = str(1)
            
            
    ET.SubElement(shipment_element, 'maxInVehicleTime').text = str(person.max_ivt)

In [11]:
from xml.dom import minidom
xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")
with open("data/sjobo_general.xml", "w") as f:
    f.write(xmlstr)

In [12]:
# tree.write('data/sjobo_integrated_vrp.xml', xml_declaration=True, encoding="UTF-8")

In [13]:
tdm = pd.DataFrame.from_dict(coordinates.coords, orient='index').reset_index().set_index(0)

In [14]:
tdm

,index
0,
0,"(13.15, 55.383333)"
1,"(13.320832, 55.6932601)"
2,"(12.782837494270069, 56.10739805)"
3,"(12.9719851, 56.2594006)"
4,"(14.15, 56.0)"
...,...
7085,"(13.7073147145297, 55.66138759817123)"
7086,"(13.83656431876258, 55.68339675499357)"
7087,"(13.76977163431982, 55.62650004347864)"


In [15]:
tdm['lon'] = tdm['index'].apply(lambda x: x[0])
tdm['lat'] = tdm['index'].apply(lambda x: x[1])

In [16]:
tdm = tdm.drop('index', axis=1)

In [17]:
tdm.to_csv('data/sjobo_general.csv', header=None)